### how to point docker CAS to an env
- flip EDMS 
    > `sed -i -e 's/aetosapp/v2/g' com.cengage.clue.pipeline.cfg`
- flip LCS
    > `sed -i -e 's/lcs-v2-int/lcs-v2-stage/g' /aetos/etc/com.cengage.lcs.client.rest.cfg`


In [17]:
def process_payload(payload):
    try:
        jsonObject = json.loads(payload)
        global final
        completed =  jsonObject.get('completed', None)
        summary = jsonObject.get('summary')
        isSbs = None
        started = None
        startTime = jsonObject.get('startTime', None)
        if summary != None :
            isSbs = summary.get('stepByStepHelpEnabled')
            started = summary.get('started', None)
            
        attemptTreePath =  jsonObject.get('attemptTreePath', None)
        
        final = final.append([{'type':jsonObject[u'type'], 
                                'id':jsonObject[u'id'],
                                'scored':jsonObject[u'scored'],
                                'cgid':jsonObject[u'cgid'],
                                'completed':completed,
                                'isSbs' : isSbs,
                                'started' : started,
                                'startTime' : startTime,
                                'attemptTreePath' : attemptTreePath,
                                'duedate':jsonObject['initParams'][u'dueDate']
                              }], ignore_index=True)
    except ValueError:
        write_to_file(payload)
        write_to_file_newline()
        print("error")

In [19]:
# brew services start cassandra@2.2 
# load the cas data into cassandra
# !pip install cassandra-driver - to install driver
#  this will create tables
#  cluster.connect('cas_a_debug') - make sure you connect to right keyspace


from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
import pandas as pd
import json

pd.set_option('display.max_colwidth', -1)
final = pd.DataFrame(columns=['type','id','scored','cgid','completed','duedate'])

def write_to_file(stringObject):
    with open('/Users/mtony/Downloads/data.json', 'a') as f:
         json.dump(stringObject, f)
            
def write_to_file_newline():
    hs = open("/Users/mtony/Downloads/data.json","a")
    hs.write("\n")
    hs.close()

ap = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(protocol_version=3, auth_provider=ap)
# cluster = Cluster(['127.0.0.1'])  # provide contact points and port
session = cluster.connect('cas_a_debug')

session.execute('CREATE TABLE IF NOT EXISTS attempt (id text PRIMARY KEY, monotonic_timestamp bigint, payload text);')
session.execute('CREATE TABLE IF NOT EXISTS componentinstance (id text PRIMARY KEY, monotonic_timestamp bigint, payload text);')
rows = session.execute('select * from attempt limit 100;')
df = pd.DataFrame(list(rows))
df = df.sort_values('monotonic_timestamp')
# df
df.payload.apply(process_payload)
# final
# final[final.scored== False]
final[(final.type == 'clue:exerciseSetQuestion')]
# .attemptTreePath
# .describe()
# final[(final.type == 'clue:exerciseSetQuestion') & (final.scored == False)]
# final.describe()
# final.type.unique()


,attemptTreePath,cgid,completed,duedate,id,isSbs,scored,startTime,started,type
7,/563ef6dc-015f-1000-0003-5fdd57b3867c/563ef9b4-015f-1000-0003-5fdd57b3867c/563efa98-015f-1000-0003-5fdd57b3867c/563efad6-015f-1000-0002-5fdd57b3867c,BBMQNLWE6J51THTEQ622,False,9223372036854775807,563efad6-015f-1000-0002-5fdd57b3867c,True,False,None,False,clue:exerciseSetQuestion
8,/563ef6dc-015f-1000-0003-5fdd57b3867c/563ef8a1-015f-1000-0003-5fdd57b3867c/563ef947-015f-1000-0000-5fdd57b3867c/563ef983-015f-1000-0003-5fdd57b3867c,WXSYT45EK3AA4499E476,False,9223372036854775807,563ef983-015f-1000-0003-5fdd57b3867c,True,False,None,False,clue:exerciseSetQuestion
18,/563ef6dc-015f-1000-0003-5fdd57b3867c/563efb08-015f-1000-0003-5fdd57b3867c/563efbdb-015f-1000-0002-5fdd57b3867c/563efc18-015f-1000-0003-5fdd57b3867c,ZKWF47LAZ93SC0NWF068,False,9223372036854775807,563efc18-015f-1000-0003-5fdd57b3867c,True,False,None,False,clue:exerciseSetQuestion
28,/563ef6dc-015f-1000-0003-5fdd57b3867c/563ef70d-015f-1000-0003-5fdd57b3867c/563ef832-015f-1000-0003-5fdd57b3867c/563ef872-015f-1000-0002-5fdd57b3867c,XXVA32GQVVMG6JMVW889,False,9223372036854775807,563ef872-015f-1000-0002-5fdd57b3867c,True,False,2017-10-25T21:15:00.873-04:00,True,clue:exerciseSetQuestion


In [21]:
# brew services start cassandra@2.2 
# load the cas data into cassandra
# !pip install cassandra-driver - to install driver
#  this will create tables
#  cluster.connect('cas_a_debug') - make sure you connect to right keyspace


from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
import pandas as pd
import json

pd.set_option('display.max_colwidth', -1)
final = pd.DataFrame(columns=['type','id','scored','cgid','completed','duedate'])

def write_to_file(stringObject):
    with open('/Users/mtony/Downloads/data.json', 'a') as f:
         json.dump(stringObject, f)
            
def write_to_file_newline():
    hs = open("/Users/mtony/Downloads/data.json","a")
    hs.write("\n")
    hs.close()
            

    

ap = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(protocol_version=3, auth_provider=ap)
# cluster = Cluster(['127.0.0.1'])  # provide contact points and port
session = cluster.connect('cas_localhost')

session.execute('CREATE TABLE IF NOT EXISTS attempt (id text PRIMARY KEY, monotonic_timestamp bigint, payload text);')
session.execute('CREATE TABLE IF NOT EXISTS componentinstance (id text PRIMARY KEY, monotonic_timestamp bigint, payload text);')
rows = session.execute('select * from attempt limit 100;')
df = pd.DataFrame(list(rows))
df = df.sort_values('monotonic_timestamp')
# df
df.payload.apply(process_payload)
# final
# final[final.scored== False]
final[(final.type == 'clue:exerciseSet')]
# .describe()

# .attemptTreePath
# final[(final.type == 'clue:exerciseSetQuestion') & (final.scored == False)]
# final.describe()
# final.type.unique()

,attemptTreePath,cgid,completed,duedate,id,isSbs,scored,startTime,started,type
31,/2c315eb4-015f-1000-0003-d66665667e6e/2c315fac-015f-1000-0003-d66665667e6e/2c315fed-015f-1000-0003-d66665667e6e,UPZMAUW1TSQ1WDKJP285,True,9223372036854775807,2c315fed-015f-1000-0003-d66665667e6e,None,True,2017-10-18T21:40:43.585-04:00,True,clue:exerciseSet
32,/2c315eb4-015f-1000-0003-d66665667e6e/2c315f01-015f-1000-0002-d66665667e6e/2c315f38-015f-1000-0000-d66665667e6e,ESHT4NAJFXRCNBHTK298,True,9223372036854775807,2c315f38-015f-1000-0000-d66665667e6e,None,True,2017-10-17T22:28:48.957-04:00,True,clue:exerciseSet
34,/2c315eb4-015f-1000-0003-d66665667e6e/2c316006-015f-1000-0003-d66665667e6e/2c31604d-015f-1000-0003-d66665667e6e,UPTFZE1GXMU1FQUBB027,False,9223372036854775807,2c31604d-015f-1000-0003-d66665667e6e,None,True,None,False,clue:exerciseSet
35,/2c315eb4-015f-1000-0003-d66665667e6e/2c315f51-015f-1000-0003-d66665667e6e/2c315f93-015f-1000-0003-d66665667e6e,CDPEFMS7C19L7Y5JN830,True,9223372036854775807,2c315f93-015f-1000-0003-d66665667e6e,None,True,2017-10-18T21:05:41.510-04:00,True,clue:exerciseSet
42,/2c315eb4-015f-1000-0003-d66665667e6e/2c315ebe-015f-1000-0003-d66665667e6e/2c315ee8-015f-1000-0003-d66665667e6e,ZMNFMZ6C8Z4U56508440,True,9223372036854775807,2c315ee8-015f-1000-0003-d66665667e6e,None,True,2017-10-17T21:50:31.973-04:00,True,clue:exerciseSet
47,/563ef6dc-015f-1000-0003-5fdd57b3867c/563ef8a1-015f-1000-0003-5fdd57b3867c/563ef947-015f-1000-0000-5fdd57b3867c,LFNE80YEGQ2KVQDMN061,False,9223372036854775807,563ef947-015f-1000-0000-5fdd57b3867c,None,False,None,False,clue:exerciseSet
48,/563ef6dc-015f-1000-0003-5fdd57b3867c/563ef9b4-015f-1000-0003-5fdd57b3867c/563efa98-015f-1000-0003-5fdd57b3867c,CDFKGFX1E4BFD99FA327,False,9223372036854775807,563efa98-015f-1000-0003-5fdd57b3867c,None,False,None,False,clue:exerciseSet
56,/563ef6dc-015f-1000-0003-5fdd57b3867c/563efb08-015f-1000-0003-5fdd57b3867c/563efbdb-015f-1000-0002-5fdd57b3867c,FYXKD0A0CW7HWTDDB778,False,9223372036854775807,563efbdb-015f-1000-0002-5fdd57b3867c,None,False,None,False,clue:exerciseSet
66,/563ef6dc-015f-1000-0003-5fdd57b3867c/563ef70d-015f-1000-0003-5fdd57b3867c/563ef832-015f-1000-0003-5fdd57b3867c,FUZVQAPKX17JCAJZC490,False,9223372036854775807,563ef832-015f-1000-0003-5fdd57b3867c,None,False,2017-10-25T21:15:00.855-04:00,True,clue:exerciseSet
90,/5c3899af-015f-1000-0000-7d2fe5eecefc/5c38a970-015f-1000-0002-7d2fe5eecefc/5c38b5e5-015f-1000-0002-7d2fe5eecefc,ESHT4NAJFXRCNBHTK298,False,9223372036854775807,5c38b5e5-015f-1000-0002-7d2fe5eecefc,None,False,2017-10-27T05:08:00.439Z,True,clue:exerciseSet
